# Downloading PlanetScope Scenes using Planet Orders API
---

**Objectives:**

By the end of this exercise, you should be able:
* to access Planet Orders API
* to search for PlanetScope Scenes given a set of rules (e.g., using a area of interest to filter out images)
* to download multiple PlanetScope Scenes using Planet Orders API
---

We will continue to use Lake Raleigh as our area of interest (AOI). Planet has multiple [APIs](https://developers.planet.com/docs/apis/) that serve different purposes. In this exercise, we will use the [Data API](https://developers.planet.com/docs/apis/data/) and the [Orders API](https://developers.planet.com/apis/orders/). The Data API provides the [Quick Search](https://developers.planet.com/docs/apis/data/quick-saved-search/) functionality, which is the easiest way to search the Planet catalog. The Orders API is used to access analysis ready data (e.g., surface reflectance imagery) and have the images delivered directly to our local or cloud storage.

The Orders API makes it easier to create pipelines to continuously downloading imagery for processing and analysis. For instance, you can manually download imagery using [Planet Explorer](https://developers.planet.com/docs/apps/explorer/) or Planet [QGIS](https://developers.planet.com/docs/integrations/qgis/) or [ArcGIS](https://developers.planet.com/docs/integrations/arcgis/); nonetheless, this quickly becomes a burden if you have to download thousands of images for multiple AOIs!   

#### Getting Started with Planet APIs

To use any Planet API, you'll need an API key. API keys are available to all registered users with active Planet accounts. Once you signed up for the [Education and Research Program](https://www.planet.com/markets/education-and-research/) account, you will need to get your API key. To do so, log in to your account at [planet.com/account](https://www.planet.com/account/). Please note it may take up to two weeks for your account to be activated under the Education and Research Program.

<p align="center">
    <img src='imgs/planet-api-key.png' width='1200' /> 
</p>

In [6]:
import os
import time
import json
import geojson
import requests
import numpy as np
import pandas as pd
from pathlib import Path

from shapely import geometry as sgeom
from requests.auth import HTTPBasicAuth

#### Authentication

You will need your Planet API Key here. The easiest way to authenticate is to simply copy and past the API key into the Jupyter Notebook—we will use this method to make it simpler. Nonetheless, this is not recommended, as this document may be public (i.e., in your GitHub account) or you may share it with others. Therefore, for future reference, it is recommended that you store your API key as an environment variable, which you can read more about [here](https://www.nylas.com/blog/making-use-of-environment-variables-in-python/). 

In [7]:
PL_API_KEY = 'enter your api key here'

# URL to access the Orders API
orders_url = 'https://api.planet.com/compute/ops/orders/v2'

To communicate with the API we will use the python package `requests`. First, we will make sure that the authentication and communication are working as expected. We expect to get a response code of `200` from this API call. To troubleshoot other response codes, please check [here](https://developers.planet.com/docs/orders/reference/#operation/listOrders).

In [8]:
auth = HTTPBasicAuth(PL_API_KEY, '')
response = requests.get(orders_url, auth=auth)
response

<Response [200]>

Now that the communication is working fine, we will build up a request to search for images. We will load our geojson (AOI for Lake Raleigh), and define certain parameters, for example, cloud cover, dates, etc.  

### Loading the coordinates (area of interest, AOI) from the lake-raleigh.geojson

In [9]:
with open('aoi/lake-raleigh.geojson') as f:
    gj = geojson.load(f)

coords = gj['features'][0]['geometry']['coordinates'][0]
aoi_geom = {"type": "Polygon", "coordinates": coords}
aoi_geom

{'type': 'Polygon',
 'coordinates': [[[-78.687957, 35.769435],
   [-78.675624, 35.769435],
   [-78.675691, 35.76139],
   [-78.688092, 35.761281],
   [-78.687957, 35.769435]]]}

### Defining the general objects for imagery search

First, we need to define a time frame for our search at Planet's catalog, this includes a starting and an ending date. Then, we will define a cloud cover threshold (from 0 to 1) with a less than or equal (lte) rule, and the type of imagery that we want. Planet has several item [types](https://developers.planet.com/docs/apis/data/items-assets/), we will be using [PSScenes](https://developers.planet.com/docs/data/psscene/), and we will download the following asset types: `'ortho_analytic_4b'` and `'ortho_udm2'`. See all available asset types [here](https://developers.planet.com/docs/data/psscene/).

In [10]:
st_date, end_date = '2021-11-14', '2021-11-15'
cloud_cover = 0.05  # 5% of cloud cover allowed
item_type = 'PSScene'
asset_one, asset_two = 'ortho_analytic_4b', 'ortho_udm2'

After defining the parameters, we need to pass them along as dictionaries to create the request body that we will send to the API. We have a total of four filters—geometry, date, cloud and asset type—in the form of dictionaries. More examples can be found [here](https://developers.planet.com/docs/apis/data/searches-filtering/).

In [11]:
geometry_filter = {"type": "GeometryFilter",
                   "field_name": "geometry",
                   "config": aoi_geom}

date_filter = {"type": "DateRangeFilter",
               "field_name": "acquired",
               "config": {"gte": "{}T00:00:00.000Z".format(st_date), "lte": "{}T23:59:59.999Z".format(end_date)}}

cloud_filter = {"type": "RangeFilter",
                "field_name": "cloud_cover",
                "config": {"lte": cloud_cover}}

asset_type = {"type": "AndFilter",
              "config": [{"type": "AssetFilter", "config": [asset_one]},
                         {"type": "AssetFilter", "config": [asset_two]}]}

combined_filter = {"type": "AndFilter",
                   "config": [geometry_filter, date_filter, cloud_filter, asset_type]}

search_request = {"item_types": [item_type],
                  "filter": combined_filter}
search_request

{'item_types': ['PSScene'],
 'filter': {'type': 'AndFilter',
  'config': [{'type': 'GeometryFilter',
    'field_name': 'geometry',
    'config': {'type': 'Polygon',
     'coordinates': [[[-78.687957, 35.769435],
       [-78.675624, 35.769435],
       [-78.675691, 35.76139],
       [-78.688092, 35.761281],
       [-78.687957, 35.769435]]]}},
   {'type': 'DateRangeFilter',
    'field_name': 'acquired',
    'config': {'gte': '2021-11-14T00:00:00.000Z',
     'lte': '2021-11-15T23:59:59.999Z'}},
   {'type': 'RangeFilter',
    'field_name': 'cloud_cover',
    'config': {'lte': 0.05}},
   {'type': 'AndFilter',
    'config': [{'type': 'AssetFilter', 'config': ['ortho_analytic_4b']},
     {'type': 'AssetFilter', 'config': ['ortho_udm2']}]}]}}

### Posting the search request
Now that we created the search request with the imagery filtering rules that we want, we will build a `post` request to interact with the API. Again, if sucessfull, we should expect `200` as the post response.

To help us with the API output, we will leverage the two functions described below: `fetch_page` and `get_meta`. The first function will allow us to paginate over our search in the API. Briefly, if the output of our search has more than 250 elements (i.e., more than 250 images based on our filters), we will have multiple pages (each page has a link). Hence, we need to get the information from the first page, applying `get_meta`, and then, we need to move on to the next page (paginate!), and we will paginate using `fetch_page`. In case we have less than 250 elements per search, then we only have one page, therefore, we do not need to paginate.

As a last step, we will get the geometry (extent) of the image using `shapely_geom` as a shapely object. This is necessary to calculate the percentage of overlap betweent the image and our AOI.

Do not worry for now if you do not understand the 3 functions below.

In [12]:
def get_meta(page_info):
    """Transforms request .json response into pd.DataFrame() and get the images metadata information.
    """
    frame = pd.DataFrame([{**{'image_id': img['id']}, **img['properties']} for img in page_info['features']])
    return frame


def shapely_geom(geom):
    """Converts the AOI geometry to a shapely structure.
    """
    aoi = {u'geometry': {u'type': u'Polygon', u'coordinates': geom['coordinates']}}
    aoi_shape = sgeom.shape(aoi['geometry'])
    return aoi_shape


def fetch_page(url, _list):
    """Paginates over API request if more than 250 results are found on Planet Catalog.
    """
    s = requests.get(url, auth=auth)
    res_code = s.status_code

    while res_code == 429:
        print('rate of requests too high! sleep 2s...')
        time.sleep(2)
        s = requests.get(url, auth=auth)
        res_code = s.status_code

    result = s.json()
    metadata = get_meta(result)
    metadata['geom'] = [shapely_geom(geom['geometry']) for geom in result['features']]
    _list.append(metadata)

    next_url = result["_links"].get("_next")

    if next_url:
        fetch_page(next_url, _list)

We are now building the post request to interact with the API.

In [13]:
url_quick_search = "https://api.planet.com/data/v1/quick-search"  # url to search for imagery
res = requests.post(url_quick_search, auth=(PL_API_KEY, ''), json=search_request)

if not res.status_code == 200:
    print(res.text)
else:
    # get results from post and link of the 1st page
    post_result = res.json()
    link_first_page = post_result['_links']['_first']

    _meta = []
    fetch_page(link_first_page, _meta)  # paginates if necessary and stores metadata to _meta

    if len(_meta) > 0:
        metadata = pd.concat(_meta)
    else:
        metadata = _meta[0]

    image_ids = metadata.image_id.tolist()

    if len(image_ids) == 0:
        print("No suitable images were found. Double check filters, including asset_types.")
    else:
        print("Found {} image(s) matching the filters.".format(len(image_ids)))
        metadata.head()

Found 1 image(s) matching the filters.


### Refining our search

When using Planet Explorer, we are able to set an AOI coverage (%) filter, which tells us how much of the AOI is covered by the image. This is very important because it enables us to filter out images that cover only a small part of our AOI. For instance, see the example below, in which only the blue shaded part of the AOI is covered by the image. The Orders API does not have a built in filter to only select images that overlap with a large part of our AOI, therefore, we will create our own method to filter out those images.

We will use the image metadata from our search to calculate the percentage of overlap between the image extent and the AOI extent. 

<p align="center">
    <img src='imgs/planet-aoi-coverage.png' width='1200' /> 
</p>

In [14]:
# transform aoi_geom to shapely shape
aoi_shape = shapely_geom(aoi_geom)

_cov = []
for index, image_shape in enumerate(metadata.geom.tolist()):
    coverage = np.round((aoi_shape.intersection(image_shape).area / aoi_shape.area) * 100, 2)
    _cov.append(coverage)

metadata['coverage'] = _cov
metadata.head()

,image_id,acquired,anomalous_pixels,clear_confidence_percent,clear_percent,cloud_cover,cloud_percent,ground_control,gsd,heavy_haze_percent,...,snow_ice_percent,strip_id,sun_azimuth,sun_elevation,updated,view_angle,visible_confidence_percent,visible_percent,geom,coverage
0,20211114_160039_40_2402,2021-11-14T16:00:39.405833Z,0.0,99.0,100.0,0.0,0.0,True,4.0,0.0,...,0.0,5098663,162.6,33.9,2021-11-16T17:32:46Z,4.1,73.0,100.0,POLYGON ((-78.96215928689067 35.92796872381647...,100.0


In [15]:
coverage_thresh = 90
images_subset = metadata[metadata.coverage >= coverage_thresh]
print(f'After filtering out images that did not cover {coverage_thresh}% or more of our AOI, were are left with {len(images_subset)} image(s).')

After filtering out images that did not cover 90% or more of our AOI, were are left with 1 image(s).


We also have images that were collected on the same day, within minutes or hours apart. After we applied our AOI coverage filter, all images that are left cover >= 90% of our AOI. In this case, we could use another set of rules, based on the images' metadata, to decide which images to chose from those that were collected on the same day. To make it simpler, we will choose the image that was collected first, using the `'acquired'` column from `images_subset`.

In [16]:
dates = [d[0:10] for d in images_subset.acquired.tolist()]
images_subset = images_subset.assign(dates=dates)  # create new column called 'dates' using assign

# drop duplicates using column 'dates'
images_subset = images_subset.drop_duplicates(subset='dates', keep='first')
print(f'After filtering out images that were collected on the same day, were are left with {len(images_subset)} image(s).')

After filtering out images that were collected on the same day, were are left with 1 image(s).


### Place imagery order, activate items and download imagery

To download the images, we need their image id, which is available within the metadata.

In [20]:
# general objects
image_ids = images_subset.image_id.tolist()  # images that will be downloaded
url_orders = 'https://api.planet.com/compute/ops/orders/v2'  # POST
headers = {'content-type': 'application/json'}
order_name = 'lake_raleigh'
save_path = os.path.join(os.getcwd(), 'results')  # you can change if you would like to save in another folder

### Creating and posting the payload for imagery download

In [21]:
# create post payload
payload = {
    "name": order_name,
    "order_type": "partial",
    "products": [{
        "item_ids": image_ids,
        "item_type": 'PSScene',
        "product_bundle": 'analytic_sr_udm2'
    }],
    "tools": [
        {
            "harmonize": {
                "target_sensor": "Sentinel-2"
            }},
        {
            "clip": {
                "aoi": aoi_geom  # Lake Raleigh extent
            }
        }
    ],
    "delivery": {},
    "notifications": {
        "email": False
    }
}

response = requests.post(url_orders, data=json.dumps(payload), auth=(PL_API_KEY, ''), headers=headers)

if response.status_code == 202:
    print("Order succesfully placed with order id {}".format(response.json()["id"]))
    order_id = response.json()["id"]
    order_url = url_orders + "/" + order_id
    print(f'Order url: \n {order_url}')
else:
    print("Order failed with error {}".format(response.json()))

Order succesfully placed with order id ed4b1863-0d9b-4766-b20b-1a7f9519437b
Order url: 
 https://api.planet.com/compute/ops/orders/v2/ed4b1863-0d9b-4766-b20b-1a7f9519437b


### Checking the order status, polling for success, and downloading the images.
This function below will take a while to run, it took ~20 min on my laptop

In [22]:
def poll_for_success(order_url, num_loops=100):
    print("Polling order..")
    count = 0
    while count < num_loops:
        count += 1
        r = requests.get(order_url, auth=(PL_API_KEY, ''))
        response = r.json()
        state = response["state"]
        print("Current state: {}".format(state))
        end_states = ["success", "failed", "partial"]
        if state in end_states:
            results = response["_links"]["results"]
            results_name = [r["name"] for r in results]
            results_urls = [r["location"] for r in results]
            break
        time.sleep(60)

    return [results_name, results_urls]

location_url_list = poll_for_success(order_url)

Polling order..
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: success


### Downloading the images once the order is ready (current state = *'success'*).  

We will download the images and save them at the `'save_path'` directory that we defined above. The function `download_results()` will create the `'save_path'` directory if it does not exist.

In [23]:
def download_results(results_urls, results_name, overwrite=False):
    print("{} items to download".format(len(results_urls)))
    for url, name in zip(results_urls, results_name):
        path = Path(save_path + f'/images/{order_name}/{name}')
        if overwrite or not path.exists():
            print("Downloading : {}".format(os.path.basename(path)))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, "wb").write(r.content)
        else:
            print("{} :Already exists, skipping".format(os.path.basename(path)))


if len(location_url_list) > 0:
    results_names = location_url_list[0]
    results_urls = location_url_list[1]
    download_results(results_urls, results_names)

5 items to download
